In [1]:
!pip install xlsxwriter

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [3]:
import tensorflow as tf  # TensorFlow library for deep learning
# Importing various pre-trained models from Keras applications
from tensorflow.keras.applications import (VGG16, Xception, InceptionV3, ResNet50,
                                           EfficientNetB0, EfficientNetB1, EfficientNetB2,
                                           EfficientNetB3, EfficientNetB4, EfficientNetB5,
                                           EfficientNetB6, EfficientNetB7)
# Importing necessary modules from TensorFlow and Keras
from tensorflow.keras import layers, regularizers  # Layers and regularization techniques
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau  # Training callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Image data preprocessing
# Importing other essential libraries for data handling and evaluation
import numpy as np  # Numerical operations library
import os  # Operating system related functionality
from glob import glob  # File searching and pattern matching
import shutil  # High-level file operations
import zipfile  # ZIP file operations
import random  # Random number generation
import pandas as pd  # Data manipulation library
import matplotlib.pyplot as plt  # Visualization library
from tensorflow.keras.callbacks import Callback  # Base class for Keras callbacks


In [38]:
compressed_paths = glob("DataSets/Elsafty_RBCs_for_Classification.zip")

In [18]:
Save_model_at          = "Classification"  # Directory to save the trained model
Model_name             = "Classifier"  # Name for the model
Transfer_Larning_Model = EfficientNetB0  # Transfer learning model choice (EfficientNetB0, VGG16, etc.)

In [19]:
Shuffle_Before_Split   = "no"  # Flag indicating whether to shuffle data before split ("no" or "yes")

selected_split_part    = 1  # Selected part of the images to be used for testing subset
num_split_parts        = 6  # Total number of parts of the images for dataset splitting
Give_data_balance      = "no"  # Flag indicating whether to balance data ("no" or "yes")
batch_size             = 32  # Batch size for training
Epochs_number          = 20  # Number of epochs for training
initial_LearningRate   = 0.000004  # Initial learning rate for the optimizer
Patience               = 3  # Number of epochs with no improvement after which learning rate will be reduced
min_delta              = 0.01  # Minimum change in monitored quantity for reducing the learning rate
image_width            = 80  # Width of images
image_height           = 80  # Height of images

# Slides to be excluded in this experiment (used for Leave-One-Out experiments)
Slides_to_be_excluded = ["nothing to be excluded"]

In [39]:
# Calculate the number of classes and extract class names from ZIP file paths
num_classes = len(compressed_paths)
classes = [path.split('/')[-1].split('.zip')[0] for path in compressed_paths]
classes = sorted(classes)

# Define directories for training, validation, test, and dataset root
training_directory = "Classification/Training"
val_directory = "Classification/Validation"
test_directory = "Classification/Test"
root = "Classification/Decompressed_dataset/"

In [40]:
# Display configuration information
print("Model_name = ", Model_name + "_Split_Part_" + str(selected_split_part) + "_from_" + str(num_split_parts))
print("Test_split_part = ", selected_split_part, " Total_split_parts = ", num_split_parts)
print("Shuffle_Before_Split = ", Shuffle_Before_Split)
if str(Give_data_balance).lower() != "no":
    print("Give_data_balance = ", Give_data_balance)
print("Batch Size = ", batch_size)
print("Epochs_number = ", Epochs_number)
print("initial_LearningRate = ", initial_LearningRate)
print("Patience = ", Patience, " min Delta = ", min_delta)
print("Images width = ", image_width, " Images height = ", image_height)
print("num_classes = ", num_classes)

# Attempt to remove the 'root' directory if it exists
# This step removes previously extracted data
try:
    shutil.rmtree(root[:-1])
except:
    pass

Model_name =  Classifier_Split_Part_1_from_6
Test_split_part =  1  Total_split_parts =  6
Shuffle_Before_Split =  no
Batch Size =  32
Epochs_number =  20
initial_LearningRate =  4e-06
Patience =  3  min Delta =  0.01
Images width =  80  Images height =  80
num_classes =  1


In [16]:
import os
print(os.getcwd()) 

/Users/vighneshms/Downloads/Red_cell.Ai


In [41]:
# Function to check if filename contains any of the specified slides
def contains_string(filename, strings):
    for s in strings:
        if s in filename:
            return True
    return False

# Extract files from each ZIP file
for compressed_file in compressed_paths:
    with zipfile.ZipFile(compressed_file, "r") as zip_ref:
        for file_info in zip_ref.infolist():
            if not contains_string(file_info.filename, Slides_to_be_excluded):
                zip_ref.extract(file_info, path=root)

# Remove 'training_directory', 'val_directory', and 'test_directory' if they exist
# This step clears existing directories for fresh data organization
try:
    shutil.rmtree(training_directory)
except:
    pass
try:
    shutil.rmtree(val_directory)
except:
    pass
try:
    shutil.rmtree(test_directory)
except:
    pass

In [50]:
classes = [cls for cls in os.listdir(root) if os.path.isdir(os.path.join(root, cls))]

for cls in classes:
    # Define the folder containing images for the current class
    class_folder = os.path.join(root, cls)

    # List all files in the class folder and exclude directories
    allfiles = [f for f in os.listdir(class_folder) if os.path.isfile(os.path.join(class_folder, f))]
    if not allfiles:
        print(f"No files found in {class_folder}")
        continue

    # Shuffle images before splitting
    if str(Shuffle_Before_Split).lower() != "no":
        random.seed(1234)
        random.shuffle(allfiles)

    # Split files into training, validation, and test parts
    part_length = len(allfiles) // num_split_parts
    remainder = len(allfiles) % num_split_parts
    start_index = 0
    result = []

    for _ in range(num_split_parts):
        end_index = start_index + part_length
        if remainder > 0:
            end_index += 1
            remainder -= 1
        result.append(allfiles[start_index:end_index])
        start_index = end_index

    # Test, validation, and training files
    test_inx = selected_split_part - 1
    val_inx = (selected_split_part % num_split_parts)
    test_files = result[test_inx]
    val_files = result[val_inx]
    train_files = [x for x in allfiles if x not in test_files + val_files]

    # Create directories for training, validation, and test
    for directory in [training_directory, val_directory, test_directory]:
        os.makedirs(os.path.join(directory, cls), exist_ok=True)

    # Copy files
    for file_group, dest_dir in [(test_files, test_directory), (val_files, val_directory), (train_files, training_directory)]:
        for file in file_group:
            src = os.path.join(class_folder, file)
            dest = os.path.join(dest_dir, cls, file)
            try:
                shutil.copy(src, dest)
            except Exception as e:
                print(f"Error copying {file} for class {cls}: {e}")

    print(f"Class '{cls}' processing complete.")

Class 'Elsafty_RBCs_for_Classification' processing complete.


In [61]:
import os
import shutil

# Paths
source_root = "Classification/Decompressed_dataset/Elsafty_RBCs_for_Classification"  # Source folder containing class folders
destination_root = "Organized_Dataset"  # New root folder for organized dataset

# Subdirectories for organized dataset
training_directory = os.path.join(destination_root, "Training")
validation_directory = os.path.join(destination_root, "Validation")
test_directory = os.path.join(destination_root, "Test")

# Create destination directories
for directory in [training_directory, validation_directory, test_directory]:
    os.makedirs(directory, exist_ok=True)

# Configuration
num_split_parts = 6  # Number of parts to split the dataset into
selected_split_part = 1  # Part to use as the test set
shuffle_before_split = True  # Whether to shuffle files before splitting

# Get class folders
classes = [cls for cls in os.listdir(source_root) if os.path.isdir(os.path.join(source_root, cls))]

for cls in classes:
    # Define paths for the current class
    class_folder = os.path.join(source_root, cls)

    # Get all files in the class folder
    all_files = [f for f in os.listdir(class_folder) if os.path.isfile(os.path.join(class_folder, f))]
    if not all_files:
        print(f"No files found in {class_folder}, skipping...")
        continue

    # Shuffle files if needed
    if shuffle_before_split:
        import random
        random.seed(1234)
        random.shuffle(all_files)

    # Split files into parts
    part_length = len(all_files) // num_split_parts
    remainder = len(all_files) % num_split_parts
    parts = []
    start_index = 0

    for _ in range(num_split_parts):
        end_index = start_index + part_length + (1 if remainder > 0 else 0)
        remainder -= 1 if remainder > 0 else 0
        parts.append(all_files[start_index:end_index])
        start_index = end_index

    # Determine test, validation, and training files
    test_files = parts[selected_split_part - 1]
    validation_files = parts[selected_split_part % num_split_parts]
    training_files = [f for f in all_files if f not in test_files + validation_files]

    # Copy files to respective directories
    for file_group, target_directory in [
        (test_files, test_directory),
        (validation_files, validation_directory),
        (training_files, training_directory),
    ]:
        class_target_dir = os.path.join(target_directory, cls)
        os.makedirs(class_target_dir, exist_ok=True)
        for file in file_group:
            src = os.path.join(class_folder, file)
            dst = os.path.join(class_target_dir, file)
            shutil.copy(src, dst)

    print(f"Class '{cls}' processing complete. Training: {len(training_files)}, Validation: {len(validation_files)}, Test: {len(test_files)}")

print(f"Files organized in {destination_root}")


Class 'SEGMENTED - Class 7 - Teardrops' processing complete. Training: 10864, Validation: 2717, Test: 2717
Class 'SEGMENTED - Class 6 - Burr Cells' processing complete. Training: 5964, Validation: 1492, Test: 1492
Class 'SEGMENTED - Class 5 - Three Overlapping RBCs' processing complete. Training: 10384, Validation: 2596, Test: 2597
Class 'SEGMENTED - Class 2 - Ovalocytes' processing complete. Training: 36715, Validation: 9179, Test: 9179
Class 'SEGMENTED - Class 8 - Angled Cells' processing complete. Training: 16124, Validation: 4031, Test: 4032
Class 'SEGMENTED - Class 1 - Rounded RBCs' processing complete. Training: 30892, Validation: 7723, Test: 7723
Class 'SEGMENTED - Class 3 - Fragmented RBCs' processing complete. Training: 4790, Validation: 1198, Test: 1198
Class 'SEGMENTED - Class 9 - Borderline Ovalocytes' processing complete. Training: 23692, Validation: 5924, Test: 5924
Class 'SEGMENTED - Class 4 - Two Overlapping RBCs' processing complete. Training: 20906, Validation: 5227, 

In [43]:
for cls in classes:
    # Create directories for each class within 'training_directory', 'val_directory', and 'test_directory'
    os.makedirs(os.path.join(training_directory + "/", cls), exist_ok=True)
    os.makedirs(os.path.join(val_directory + "/", cls), exist_ok=True)
    os.makedirs(os.path.join(test_directory + "/", cls), exist_ok=True)

    # Define the folder containing images for the current class
    class_folder = os.path.join(root, cls)

    # List all files in the class folder and shuffle their order
    allfiles = os.listdir(class_folder)

    # Shuffle images before splitting
    if str(Shuffle_Before_Split).lower() != "no":
        random.seed(1234)
        random.shuffle(allfiles)

    # Divide images into parts
    part_length = len(allfiles) // num_split_parts
    remainder = len(allfiles) % num_split_parts
    start_index = 0
    result = []

    # Create split parts with approximately equal sizes
    for _ in range(num_split_parts):
        end_index = start_index + part_length
        if remainder > 0:
            end_index += 1
            remainder -= 1
        result.append(allfiles[start_index:end_index])
        start_index = end_index

    # Determine test and validation indices based on selected part
    test_inx = selected_split_part - 1
    val_inx = selected_split_part
    if val_inx >= num_split_parts:
        val_inx = 0
    test_files = result[test_inx]
    val_files = result[val_inx]

    # Separate files for training
    new_lst = [x for x in allfiles if x not in test_files]
    train_files = [x for x in new_lst if x not in val_files]

    # Copy images to respective directories for training, validation, and test
    for file in test_files:
        shutil.copy(os.path.join(class_folder, file), os.path.join(test_directory + "/", cls, file))
    for file in val_files:
        shutil.copy(os.path.join(class_folder, file), os.path.join(val_directory + "/", cls, file))
    for file in train_files:
        shutil.copy(os.path.join(class_folder, file), os.path.join(training_directory + "/", cls, file))
    print("bring class", cls)  # Print the class name to indicate progress





IsADirectoryError: [Errno 21] Is a directory: 'Classification/Decompressed_dataset/Elsafty_RBCs_for_Classification/SEGMENTED - Class 7 - Teardrops'

In [47]:
for cls in classes:
    # Define the folder containing images for the current class
    class_folder = os.path.join(root, cls)
    if not os.path.exists(class_folder):
        print(f"Directory not found: {class_folder}")
        continue

    # List all files in the class folder and exclude directories
    allfiles = [f for f in os.listdir(class_folder) if os.path.isfile(os.path.join(class_folder, f))]
    if not allfiles:
        print(f"No files found in {class_folder}")
        continue

    # Shuffle images before splitting
    if str(Shuffle_Before_Split).lower() != "no":
        random.seed(1234)
        random.shuffle(allfiles)

    # Split files into training, validation, and test parts
    part_length = len(allfiles) // num_split_parts
    remainder = len(allfiles) % num_split_parts
    start_index = 0
    result = []

    for _ in range(num_split_parts):
        end_index = start_index + part_length
        if remainder > 0:
            end_index += 1
            remainder -= 1
        result.append(allfiles[start_index:end_index])
        start_index = end_index

    # Test, validation, and training files
    test_inx = selected_split_part - 1
    val_inx = selected_split_part % num_split_parts
    test_files = result[test_inx]
    val_files = result[val_inx]
    train_files = [x for x in allfiles if x not in test_files + val_files]

    # Copy images to respective directories
    for file in test_files:
        shutil.copy(os.path.join(class_folder, file), os.path.join(test_directory, cls, file))
    for file in val_files:
        shutil.copy(os.path.join(class_folder, file), os.path.join(val_directory, cls, file))
    for file in train_files:
        shutil.copy(os.path.join(class_folder, file), os.path.join(training_directory, cls, file))

    print(f"Class '{cls}' processing complete.")


Class 'Elsafty_RBCs_for_Classification' processing complete.


In [64]:
print("Training directory path:", training_directory)
print("Validation directory path:", validation_directory)
print("Test directory path:", test_directory)


Training directory path: Organized_Dataset/Training
Validation directory path: Organized_Dataset/Validation
Test directory path: Organized_Dataset/Test


In [65]:
import os

# Function to count all files in a directory, including subdirectories
def count_files(folder_path):
    count = 0
    for root, dirs, allfiles in os.walk(folder_path):
        print(f"Inspecting {root}: {len(allfiles)} files found")  # Debugging line
        count += len(allfiles)
    return count

# Updated directories
training_directory = "Classification/Training"
validation_directory = "Classification/Validation"
test_directory = "Classification/Test"

# Debugging: Print directory paths
print("Training directory path:", training_directory)
print("Validation directory path:", validation_directory)
print("Test directory path:", test_directory)

# Calculate and print the number of files in the training directory
training_file_count = count_files(training_directory)
print("Training allfiles =", training_file_count)

# Calculate and print the number of files in the validation directory
validation_file_count = count_files(validation_directory)
print("Validation allfiles =", validation_file_count)

# Calculate and print the number of files in the test directory
test_file_count = count_files(test_directory)
print("Test allfiles =", test_file_count)


Training directory path: Classification/Training
Validation directory path: Classification/Validation
Test directory path: Classification/Test
Inspecting Classification/Training: 0 files found
Inspecting Classification/Training/SEGMENTED - Class 7 - Teardrops: 10864 files found
Inspecting Classification/Training/SEGMENTED - Class 6 - Burr Cells: 5964 files found
Inspecting Classification/Training/SEGMENTED - Class 5 - Three Overlapping RBCs: 10384 files found
Inspecting Classification/Training/SEGMENTED - Class 2 - Ovalocytes: 36715 files found
Inspecting Classification/Training/SEGMENTED - Class 8 - Angled Cells: 16124 files found
Inspecting Classification/Training/SEGMENTED - Class 1 - Rounded RBCs: 30892 files found
Inspecting Classification/Training/SEGMENTED - Class 3 - Fragmented RBCs: 4790 files found
Inspecting Classification/Training/SEGMENTED - Class 9 - Borderline Ovalocytes: 23692 files found
Inspecting Classification/Training/SEGMENTED - Class 4 - Two Overlapping RBCs: 209

In [66]:
def count_files(folder_path):
    count = 0
    for root, dirs, allfiles in os.walk(folder_path):
        count += len(allfiles)
    return count

# Calculate and print the number of files in the training directory
folder_path = training_directory
file_count = count_files(folder_path)
print("Training allfiles =", file_count)

# Calculate and print the number of files in the validation directory
folder_path = val_directory
file_count = count_files(folder_path)
print("Validation allfiles =", file_count)

# Calculate and print the number of files in the test directory
folder_path = test_directory
file_count = count_files(folder_path)
print("Test allfiles =", file_count)

Training allfiles = 160331
Validation allfiles = 40087
Test allfiles = 40089


In [67]:
# Define the size of the images
image_size = (image_height, image_width)
# Seed for data reproducibility
seed = 123

# Define data augmentation for the training set
train_datagen = ImageDataGenerator(
    rotation_range=360,  # Rotate images up to 360 degrees
    horizontal_flip=True,  # Flip images horizontally
    vertical_flip=True  # Flip images vertically
)

# Data generators for the validation, test, and full data sets are initialized without augmentation
val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()
fullData_datagen = ImageDataGenerator()

# Generating data batches from the directory for training, validation, and test sets
train_generator = train_datagen.flow_from_directory(
    training_directory,  # Directory containing training images
    target_size=image_size,  # Resize images to match 'image_size'
    batch_size=batch_size,  # Number of samples per gradient update
    class_mode='sparse',  # Mode for class labels (integer)
    seed=seed  # Seed for shuffling
)

validation_generator = val_datagen.flow_from_directory(
    val_directory,  # Directory containing validation images
    target_size=image_size,
    batch_size=batch_size,
    class_mode='sparse'
)

test_generator = test_datagen.flow_from_directory(
    test_directory,  # Directory containing test images
    target_size=image_size,
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False  # Do not shuffle the test data during inference
)

# Use EfficientNetB0 as the base model for transfer learning
base_model = tf.keras.applications.EfficientNetB0(
    weights='imagenet',  # Initialize with pre-trained ImageNet weights
    include_top=False,  # Exclude the fully connected layers from the top
    input_shape=(image_height, image_width, 3)  # Define input image dimensions and channels
)

# Freeze the pre-trained layers to prevent updating during training
for layer in base_model.layers:
    layer.trainable = True  # Setting the layers to trainable

# Add a custom classification head on top of the pre-trained model
preprocess_input = tf.keras.applications.efficientnet.preprocess_input  # Preprocessing input function for EfficientNet
inputs = tf.keras.Input(shape=(image_height, image_width, 3))  # Define input shape
x = preprocess_input(inputs)  # Preprocess the input
x = base_model(x, training=False)  # Utilize the base model without training it
x = layers.GlobalAveragePooling2D()(x)  # Global average pooling layer
x = tf.keras.layers.Dropout(0.2)(x)  # Dropout layer with dropout rate of 0.2
outputs = tf.keras.layers.Dense(num_classes)(x)  # Dense layer for classification
model = tf.keras.Model(inputs, outputs)  # Create the final model

# Display the model summary
print(model.summary())

Found 160331 images belonging to 9 classes.
Found 40087 images belonging to 9 classes.
Found 40089 images belonging to 9 classes.


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 80, 80, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 3, 3, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,050,852 (15.45 MB)

 Trainable params: 4,008,829 (15.29 MB)

 Non-trainable params: 42,023 (164.16 KB)

None


In [68]:
# Compile the model with specified optimizer, loss, and metrics
optimizer = tf.keras.optimizers.Adam(learning_rate=initial_LearningRate)
model.compile(
    optimizer=optimizer,
    loss=[
        tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Sparse categorical cross-entropy loss
    ],
    metrics=[
        tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')  # Sparse categorical accuracy metric
    ]
)

save_model_after_each_epoch = tf.keras.callbacks.ModelCheckpoint(
    (str(Save_model_at) + "/" + str(Model_name) + "_Split_Part_" + str(selected_split_part) + "_from_" + str(num_split_parts) + '.keras'),
    verbose=1,
    save_best_only=True  # Save only the best model
)

# Save the model after each epoch if it performs better
# save_model_after_each_epoch = tf.keras.callbacks.ModelCheckpoint(
#     (str(Save_model_at) + "/" + str(Model_name) + "_Split_Part_" + str(selected_split_part) + "_from_" + str(num_split_parts) + '.h5'),
#     verbose=1,
#     save_best_only=True  # Save only the best model
# )

# Define the learning rate reduction callback based on validation loss
lr_reduction = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=Patience,  # Number of epochs with no improvement after which learning rate will be reduced
    verbose=1,
    min_delta=min_delta  # Minimum change in the monitored quantity to qualify as an improvement
)

# Compute class weights based on the training dataset distribution
class_counts = np.bincount(train_generator.classes)  # Count occurrences of each class in the training set
total_samples = sum(class_counts)  # Calculate the total number of samples
class_weights = [total_samples / (len(classes) * count) for count in class_counts]  # Compute class weights
class_weights_dict = {class_label: weight for class_label, weight in enumerate(class_weights)}  # Create a dictionary of class weights

# Check if data balancing is required and print class counts and weights if 'yes'
if str(Give_data_balance).lower() == "yes":
    print(class_counts)
    print(class_weights)

# Train the model using the configured generators and settings
steps_per_epoch = train_generator.samples // train_generator.batch_size  # Calculate steps per epoch
validation_steps = validation_generator.samples // validation_generator.batch_size  # Calculate validation steps

In [54]:
# Compile the model with specified optimizer, loss, and metrics
optimizer = tf.keras.optimizers.Adam(learning_rate=initial_LearningRate)
model.compile(
    optimizer=optimizer,
    loss=[
        tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Sparse categorical cross-entropy loss
    ],
    metrics=[
        tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')  # Sparse categorical accuracy metric
    ]
)

# Save the model after each epoch if it performs better
save_model_after_each_epoch = tf.keras.callbacks.ModelCheckpoint(
    (str(Save_model_at) + "/" + str(Model_name) + "_Split_Part_" + str(selected_split_part) + "_from_" + str(num_split_parts) + '.h5'),
    verbose=1,
    save_best_only=True  # Save only the best model
)

# Define the learning rate reduction callback based on validation loss
lr_reduction = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=Patience,  # Number of epochs with no improvement after which learning rate will be reduced
    verbose=1,
    min_delta=min_delta  # Minimum change in the monitored quantity to qualify as an improvement
)

# Compute class weights based on the training dataset distribution
class_counts = np.bincount(train_generator.classes)  # Count occurrences of each class in the training set
total_samples = sum(class_counts)  # Calculate the total number of samples
class_weights = [total_samples / (len(classes) * count) for count in class_counts]  # Compute class weights
class_weights_dict = {class_label: weight for class_label, weight in enumerate(class_weights)}  # Create a dictionary of class weights

# Check if data balancing is required and print class counts and weights if 'yes'
if str(Give_data_balance).lower() == "yes":
    print(class_counts)
    print(class_weights)

# Train the model using the configured generators and settings
steps_per_epoch = train_generator.samples // train_generator.batch_size  # Calculate steps per epoch
validation_steps = validation_generator.samples // validation_generator.batch_size  # Calculate validation steps


ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=Classification/Classifier_Split_Part_1_from_6.h5

In [72]:
# Define TensorBoard
class CustomTensorBoardCallback_Accuracy(Callback):
    def __init__(self, metrics):
        super(CustomTensorBoardCallback_Accuracy, self).__init__()
        # Initializing the callback with a list of metrics to monitor
        self.metrics = metrics
        # Creating empty lists to store metric values, epochs, and learning rates
        self.metric_values = {metric: [] for metric in metrics}
        self.epochs = []
        self.learning_rates = []

    def on_train_begin(self, logs=None):
        pass  # No action needed when the training starts

    def on_epoch_begin(self, epoch, logs=None):
        # Adding learning rates at the beginning of each epoch
        self.learning_rates.append(self.model.optimizer.learning_rate.numpy())

    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
        self.epochs.append(epoch + 1)  # Incrementing the epoch number by 1
        for metric in self.metrics:
            # Collecting metric values at the end of each epoch
            self.metric_values[metric].append(logs.get(metric))

        # Plotting metrics and learning rates after each epoch
        fig, ax1 = plt.subplots(figsize=(10, 6))  # Creating a plot figure
        lines = []
        colors = ['blue', 'orange', 'green', 'red']  # Colors for different metrics

        # Plotting each metric curve
        for i, metric in enumerate(self.metrics):
            line, = ax1.plot(self.epochs, self.metric_values[metric], '-', color=colors[i], label=metric)
            lines.append(line)

        # Adding a line for the learning rate
        lr_line, = ax1.plot(self.epochs, self.learning_rates, '--', label='Learning Rate')

        # Setting up plot labels, title, ticks, and legend
        ax1.set_ylabel('Value')
        ax1.set_title('Epoch')
        ax1.set_xticks(np.arange(min(self.epochs), max(self.epochs) + 1, 1))
        ax1.set_ylim(0, 1.0)
        ax1.set_yticks(np.arange(0, 1.1, 0.05))
        ax1.legend(lines + [lr_line], self.metrics + ['Learning Rate'])
        ax1.grid(True)
        ax1.tick_params(axis='x', top=True, bottom=False, labeltop=True, labelbottom=False)

        # Adding a secondary y-axis for better readability
        ax2 = ax1.twinx()
        ax2.set_ylim(0, 1.0)
        ax2.set_yticks(np.arange(0, 1.1, 0.05))
        ax2.yaxis.tick_right()
        ax2.yaxis.set_label_position("right")
        ax2.set_ylabel('Value')

        # Adding text labels for curves with an offset
        offset = 0.02
        for i in range(len(self.epochs)):
            x = self.epochs[i]
            y_lr = self.learning_rates[i]
            y_val_loss = self.metric_values['val_loss'][i]
            y_val_acc = self.metric_values['val_accuracy'][i]
            lr_label = f'{y_lr:.1e}'
            lr_label = (lr_label.replace("e-0", "e-").replace("0e", "e"))[2:]
            val_loss_label = str(int(y_val_loss * 100))
            val_acc_label = str(int(y_val_acc * 100))
            ax1.text(x, y_lr - (offset * 4), lr_label, ha='center', va='bottom', color=lr_line.get_color())
            ax1.text(x, y_val_loss - (offset * 3), val_loss_label, ha='center', va='bottom', color=colors[3])
            ax1.text(x, y_val_acc + offset, val_acc_label, ha='center', va='bottom', color=colors[0])
        plt.show()


In [74]:
# Define TensorBoard Callback
class CustomTensorBoardCallback_Accuracy(Callback):
    def __init__(self, metrics):
        super(CustomTensorBoardCallback_Accuracy, self).__init__()
        # Initializing the callback with a list of metrics to monitor
        self.metrics = metrics
        # Creating empty lists to store metric values, epochs, and learning rates
        self.metric_values = {metric: [] for metric in metrics}
        self.epochs = []
        self.learning_rates = []

    def on_train_begin(self, logs=None):
        pass  # No action needed when the training starts

    def on_epoch_begin(self, epoch, logs=None):
        # Correctly access learning rate in TensorFlow 2.11+
        self.learning_rates.append(self.model.optimizer.learning_rate.numpy())

    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
        self.epochs.append(epoch + 1)  # Increment the epoch number by 1
        for metric in self.metrics:
            # Collect metric values at the end of each epoch
            self.metric_values[metric].append(logs.get(metric))

        # Plotting metrics and learning rates after each epoch
        fig, ax1 = plt.subplots(figsize=(10, 6))  # Create a plot figure
        lines = []
        colors = ['blue', 'orange', 'green', 'red']  # Colors for different metrics

        # Plot each metric curve
        for i, metric in enumerate(self.metrics):
            line, = ax1.plot(self.epochs, self.metric_values[metric], '-', color=colors[i], label=metric)
            lines.append(line)

        # Add a line for the learning rate
        lr_line, = ax1.plot(self.epochs, self.learning_rates, '--', label='Learning Rate')

        # Set up plot labels, title, ticks, and legend
        ax1.set_ylabel('Value')
        ax1.set_title('Training Progress per Epoch')
        ax1.set_xticks(np.arange(min(self.epochs), max(self.epochs) + 1, 1))
        ax1.legend(lines + [lr_line], self.metrics + ['Learning Rate'])
        ax1.grid(True)

        # Add a secondary y-axis for better readability
        ax2 = ax1.twinx()
        ax2.set_ylabel('Learning Rate')
        ax2.tick_params(axis='y')

        plt.show()


In [75]:
# List of metrics to monitor during training
metrics = ['val_accuracy', 'accuracy', 'loss', 'val_loss']

# Creating an instance of the CustomTensorBoardCallback_Accuracy class
custom_tensorboard_callback_accuracy = CustomTensorBoardCallback_Accuracy(metrics)

# Train the model with or without data balancing
if str(Give_data_balance).lower() == "yes":
    # Train the model with data balancing and specified callbacks
    model.fit(
        train_generator,
        epochs=Epochs_number,
        steps_per_epoch=steps_per_epoch,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        class_weight=class_weights_dict,  # Apply class weights during training
        callbacks=[save_model_after_each_epoch, lr_reduction, custom_tensorboard_callback_accuracy]
    )
else:
    # Train the model without data balancing and specified callbacks
    model.fit(
        train_generator,
        epochs=Epochs_number,
        steps_per_epoch=steps_per_epoch,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        callbacks=[save_model_after_each_epoch, lr_reduction, custom_tensorboard_callback_accuracy]
    )

# Load the test dataset from the directory using the specified configurations
test_dataset = tf.keras.utils.image_dataset_from_directory(
    str(test_directory),
    labels='inferred',  # Automatically infer labels from directory names
    shuffle=False,  # Keep the test dataset order fixed
    image_size=image_size,
    batch_size=batch_size
)

# Make predictions on the test dataset using the trained model
rounded_predictions2 = []
file_paths = []

for images, labels in test_dataset:  # Use images and labels in batches
    predictions = model.predict(images)  # Get predictions for each batch of images
    rounded_predictions = np.argmax(predictions, axis=1)  # Get the class index with the highest probability
    rounded_predictions2.append(rounded_predictions)
    file_paths.extend(test_dataset.file_paths)  # Keep track of the file paths

rounded_predictions2 = np.concatenate(rounded_predictions2)  # Combine predictions into a single array

# Map predicted class indices to their corresponding class names
class_nameslist = [classes[i] for i in rounded_predictions2]

# Get file paths and true class labels
file_pathss = pd.DataFrame(file_paths, columns=['Path'])  # Create a DataFrame for file paths
true_labels = [path.split(os.sep)[-2] for path in file_paths]  # Extract the true class labels from file paths
True_class_list = pd.DataFrame(true_labels, columns=['True Class'])  # Create a DataFrame for true class labels

# Combine predictions and true labels into a DataFrame
classification_result = pd.concat([file_pathss, pd.DataFrame(class_nameslist, columns=['Predicted Class']), True_class_list], axis=1)

# Identify misclassified samples
misclassified_result = classification_result[classification_result['Predicted Class'] != classification_result['True Class']]

# Save misclassified samples to an Excel file
filename = f"{Save_model_at}/{Model_name}_Split_Part_{selected_split_part}_from_{num_split_parts}_TestData.xlsx"
writer = pd.ExcelWriter(filename, engine='xlsxwriter')
misclassified_result.to_excel(writer, sheet_name='Misclassified_Test_Data', index=False)
writer.save()

print(f"Misclassified samples saved to: {filename}")


Epoch 1/20


2024-11-08 11:47:32.431789: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at sparse_xent_op.cc:103 : INVALID_ARGUMENT: Received a label value of 8 which is outside the valid range of [0, 1).  Label values: 6 0 6 1 1 8 5 4 6 1 1 0 5 4 0 8 1 1 1 0 1 7 3 7 1 8 1 1 0 0 6 1
2024-11-08 11:47:32.431811: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Received a label value of 8 which is outside the valid range of [0, 1).  Label values: 6 0 6 1 1 8 5 4 6 1 1 0 5 4 0 8 1 1 1 0 1 7 3 7 1 8 1 1 0 0 6 1
	 [[{{function_node __inference_one_step_on_data_58397}}{{node compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits}}]]


InvalidArgumentError: Graph execution error:

Detected at node compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main

  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 87, in _run_code

  File "/Users/vighneshms/Library/Python/3.9/lib/python/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/Users/vighneshms/Library/Python/3.9/lib/python/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/Users/vighneshms/Library/Python/3.9/lib/python/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/Users/vighneshms/Library/Python/3.9/lib/python/site-packages/tornado/platform/asyncio.py", line 205, in start

  F

In [73]:
# List of metrics to monitor during training
metrics = ['val_accuracy', 'accuracy', 'loss', 'val_loss']

# Creating an instance of the CustomTensorBoardCallback_Accuracy class
custom_tensorboard_callback_accuracy = CustomTensorBoardCallback_Accuracy(metrics)

# Train the model with or without data balancing
if str(Give_data_balance).lower() == "yes":
    # Train the model with data balancing and specified callbacks
    model.fit(
        train_generator,
        epochs=Epochs_number,
        steps_per_epoch=steps_per_epoch,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        class_weight=class_weights_dict,  # Apply class weights during training
        callbacks=[save_model_after_each_epoch, lr_reduction, custom_tensorboard_callback_accuracy]
    )
else:
    # Train the model without data balancing and specified callbacks
    model.fit(
        train_generator,
        epochs=Epochs_number,
        steps_per_epoch=steps_per_epoch,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        callbacks=[save_model_after_each_epoch, lr_reduction, custom_tensorboard_callback_accuracy]
    )

# Load the test dataset from the directory using the specified configurations
test_dataset = tf.keras.utils.image_dataset_from_directory(
    str(test_directory),
    labels='inferred',  # Automatically infer labels from directory names
    shuffle=False,  # Keep the test dataset order fixed
    image_size=image_size,
    batch_size=batch_size
)

# Make predictions on the test dataset using the trained model
rounded_predictions2 = []
file_paths = []

for images, labels in test_dataset:  # Use images and labels in batches
    predictions = model.predict(images)  # Get predictions for each batch of images
    rounded_predictions = np.argmax(predictions, axis=1)  # Get the class index with the highest probability
    rounded_predictions2.append(rounded_predictions)
    file_paths.extend(test_dataset.file_paths)  # Keep track of the file paths

rounded_predictions2 = np.concatenate(rounded_predictions2)  # Combine predictions into a single array

# Map predicted class indices to their corresponding class names
class_nameslist = [classes[i] for i in rounded_predictions2]

# Get file paths and true class labels
file_pathss = pd.DataFrame(file_paths, columns=['Path'])  # Create a DataFrame for file paths
true_labels = [path.split(os.sep)[-2] for path in file_paths]  # Extract the true class labels from file paths
True_class_list = pd.DataFrame(true_labels, columns=['True Class'])  # Create a DataFrame for true class labels

# Combine predictions and true labels into a DataFrame
classification_result = pd.concat([file_pathss, pd.DataFrame(class_nameslist, columns=['Predicted Class']), True_class_list], axis=1)

# Identify misclassified samples
misclassified_result = classification_result[classification_result['Predicted Class'] != classification_result['True Class']]

# Save misclassified samples to an Excel file
filename = f"{Save_model_at}/{Model_name}_Split_Part_{selected_split_part}_from_{num_split_parts}_TestData.xlsx"
writer = pd.ExcelWriter(filename, engine='xlsxwriter')
misclassified_result.to_excel(writer, sheet_name='Misclassified_Test_Data', index=False)
writer.save()

print(f"Misclassified samples saved to: {filename}")


Epoch 1/20


/Users/vighneshms/Library/Python/3.9/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-11-08 11:45:29.886234: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at sparse_xent_op.cc:103 : INVALID_ARGUMENT: Received a label value of 8 which is outside the valid range of [0, 1).  Label values: 8 1 1 0 0 0 1 0 8 3 1 8 7 0 1 8 1 1 1 3 1 1 7 2 6 8 6 2 6 7 1 7
2024-11-08 11:45:29.886275: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Received a label value of 8 which is outside the valid range of [0, 1).  Label values: 8 1 1 0 0 0 1 0 8 3 1 8 7 0 1 8 1 1 1 3 1 1 7 2 6 8 6 2 6 7 1 7
	 [[{{function_node __in

InvalidArgumentError: Graph execution error:

Detected at node compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main

  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 87, in _run_code

  File "/Users/vighneshms/Library/Python/3.9/lib/python/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/Users/vighneshms/Library/Python/3.9/lib/python/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/Users/vighneshms/Library/Python/3.9/lib/python/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/Users/vighneshms/Library/Python/3.9/lib/python/site-packages/tornado/platform/asyncio.py", line 205, in start

  F

In [70]:
# List of metrics to monitor during training
metrics = ['val_accuracy', 'accuracy', 'loss', 'val_loss']

# Creating an instance of the CustomTensorBoardCallback_Accuracy class
custom_tensorboard_callback_accuracy = CustomTensorBoardCallback_Accuracy(metrics)

# Train the model with or without data balancing
if str(Give_data_balance).lower() == "yes":
    # Train the model with data balancing and specified callbacks
    model.fit(
        train_generator,
        epochs=Epochs_number,
        steps_per_epoch=steps_per_epoch,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        class_weight=class_weights_dict,  # Apply class weights during training
        callbacks=[save_model_after_each_epoch, lr_reduction, custom_tensorboard_callback_accuracy]
    )
else:
    # Train the model without data balancing and specified callbacks
    model.fit(
        train_generator,
        epochs=Epochs_number,
        steps_per_epoch=steps_per_epoch,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        callbacks=[save_model_after_each_epoch, lr_reduction, custom_tensorboard_callback_accuracy]
    )

# Load the test dataset from the directory using the specified configurations
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    str(test_directory) + "/",
    labels=None,
    shuffle=False,
    image_size=image_size,
    batch_size=batch_size
)

# Make predictions on the test dataset using the trained model
rounded_predictions2 = []
for image in test_dataset:
    predictions = model.predict(image)  # Get predictions for each image in the test dataset
    rounded_predictions = np.argmax(predictions, axis=1)  # Get the class index with the highest probability
    rounded_predictions2.append(rounded_predictions)  # Append the predictions to a list
rounded_predictions2 = np.concatenate(rounded_predictions2)  # Concatenate all predictions into a single array

# Map predicted class indices to their corresponding class names
class_nameslist = []
for i in rounded_predictions2:
    class_nameslist.append(classes[i])  # Map the predicted indices to their respective class names

# Get file paths and true class labels for the test dataset
XX = test_dataset.file_paths
file_pathss = pd.DataFrame(XX, columns=['Path'])  # Create a DataFrame for file paths
unique_word_list = []
for path in XX:
    path = path.replace("/", "\\")
    unique_word = path.split("\\")[-2]  # Extract the true class label from the file path
    unique_word_list.append(unique_word)  # Append true class labels to a list
True_class_lsit = pd.DataFrame(unique_word_list, columns=['True Class'])  # Create a DataFrame for true class labels


AttributeError: 'Adam' object has no attribute 'lr'

In [ ]:
# Create DataFrames for predicted and true class labels along with file paths
class_nameslist = pd.DataFrame(class_nameslist, columns=['Predicted Class'])
Classification_result = pd.concat([file_pathss, class_nameslist, True_class_lsit], axis=1)

# Identify misclassified samples by comparing predicted and true class labels
misclassified_result = Classification_result[
    Classification_result['Predicted Class'] != Classification_result['True Class']]

# Save misclassified samples to an Excel file
filename = (str(Save_model_at) + "/" + str(Model_name) + "_Split_Part_" + str(selected_split_part) + "_from_" + str(
    num_split_parts) + "_TestData.xlsx")
writer = pd.ExcelWriter(filename, engine='xlsxwriter')
misclassified_result.to_excel(writer, sheet_name='Misclassified_Test_Data', index=False)  # Write misclassified data to Excel
writer.save()  # Save the Excel file

files.download(filename)  # Download the Excel file
print("************************ Test Dataset classification is completed and saved.")  # Print completion message

In [15]:
# Organize images into training, validation, and test directories.
# Iterate through each class and distribute images based on the specified number of split parts.
### Please note that on selecting the option "Shuffle_Before_Split", the total images of each class in its folder will be randomly shuffled.
### Subsequently, these shuffled images will be divided into a number of fully separated parts.
### One-part will be allocated to testing, the second to validation, and the remaining part(s) to training.
### Therefore, no cell will be present in more than one-part, and each part will contain images from each slide/patient.
### Moreover, the shuffling will be performed with a fixed seed "order" to ensure consistency when choosing different parts for testing and validation.
### Thus, the code will split the dataset with no data mix-up or allocating images from certain slides/patients to be specific for certain subset.
### This method is useful for exploring data consistency but is not ideal for generalizing performance.
### Conversely, without shuffling, the splitting will result in better performance generalization because validation and testing will be conducted on different cases.
for cls in classes:
    # Create directories for each class within 'training_directory', 'val_directory', and 'test_directory'
    os.makedirs(os.path.join(training_directory + "/", cls), exist_ok=True)
    os.makedirs(os.path.join(val_directory + "/", cls), exist_ok=True)
    os.makedirs(os.path.join(test_directory + "/", cls), exist_ok=True)

    # Define the folder containing images for the current class
    class_folder = os.path.join(root, cls)

    # List all files in the class folder and shuffle their order
    allfiles = os.listdir(class_folder)

    # Shuffle images before splitting
    if str(Shuffle_Before_Split).lower() != "no":
        random.seed(1234)
        random.shuffle(allfiles)

    # Divide images into parts
    part_length = len(allfiles) // num_split_parts
    remainder = len(allfiles) % num_split_parts
    start_index = 0
    result = []

    # Create split parts with approximately equal sizes
    for _ in range(num_split_parts):
        end_index = start_index + part_length
        if remainder > 0:
            end_index += 1
            remainder -= 1
        result.append(allfiles[start_index:end_index])
        start_index = end_index

    # Determine test and validation indices based on selected part
    test_inx = selected_split_part - 1
    val_inx = selected_split_part
    if val_inx >= num_split_parts:
        val_inx = 0
    test_files = result[test_inx]
    val_files = result[val_inx]

    # Separate files for training
    new_lst = [x for x in allfiles if x not in test_files]
    train_files = [x for x in new_lst if x not in val_files]

    # Copy images to respective directories for training, validation, and test
    for file in test_files:
        shutil.copy(os.path.join(class_folder, file), os.path.join(test_directory + "/", cls, file))
    for file in val_files:
        shutil.copy(os.path.join(class_folder, file), os.path.join(val_directory + "/", cls, file))
    for file in train_files:
        shutil.copy(os.path.join(class_folder, file), os.path.join(training_directory + "/", cls, file))
    print("bring class", cls)  # Print the class name to indicate progress

# Function to count the number of files in a directory
def count_files(folder_path):
    count = 0
    for root, dirs, allfiles in os.walk(folder_path):
        count += len(allfiles)
    return count

# Calculate and print the number of files in the training directory
folder_path = training_directory
file_count = count_files(folder_path)
print("Training allfiles =", file_count)

# Calculate and print the number of files in the validation directory
folder_path = val_directory
file_count = count_files(folder_path)
print("Validation allfiles =", file_count)

# Calculate and print the number of files in the test directory
folder_path = test_directory
file_count = count_files(folder_path)
print("Test allfiles =", file_count)

# Display a separator for clarity
print("#######################################################################")

# Define the size of the images
image_size = (image_height, image_width)
# Seed for data reproducibility
seed = 123

# Define data augmentation for the training set
train_datagen = ImageDataGenerator(
    rotation_range=360,  # Rotate images up to 360 degrees
    horizontal_flip=True,  # Flip images horizontally
    vertical_flip=True  # Flip images vertically
)

# Data generators for the validation, test, and full data sets are initialized without augmentation
val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()
fullData_datagen = ImageDataGenerator()

# Generating data batches from the directory for training, validation, and test sets
train_generator = train_datagen.flow_from_directory(
    training_directory,  # Directory containing training images
    target_size=image_size,  # Resize images to match 'image_size'
    batch_size=batch_size,  # Number of samples per gradient update
    class_mode='sparse',  # Mode for class labels (integer)
    seed=seed  # Seed for shuffling
)

validation_generator = val_datagen.flow_from_directory(
    val_directory,  # Directory containing validation images
    target_size=image_size,
    batch_size=batch_size,
    class_mode='sparse'
)

test_generator = test_datagen.flow_from_directory(
    test_directory,  # Directory containing test images
    target_size=image_size,
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False  # Do not shuffle the test data during inference
)

# Use EfficientNetB0 as the base model for transfer learning
base_model = tf.keras.applications.EfficientNetB0(
    weights='imagenet',  # Initialize with pre-trained ImageNet weights
    include_top=False,  # Exclude the fully connected layers from the top
    input_shape=(image_height, image_width, 3)  # Define input image dimensions and channels
)

# Freeze the pre-trained layers to prevent updating during training
for layer in base_model.layers:
    layer.trainable = True  # Setting the layers to trainable

# Add a custom classification head on top of the pre-trained model
preprocess_input = tf.keras.applications.efficientnet.preprocess_input  # Preprocessing input function for EfficientNet
inputs = tf.keras.Input(shape=(image_height, image_width, 3))  # Define input shape
x = preprocess_input(inputs)  # Preprocess the input
x = base_model(x, training=False)  # Utilize the base model without training it
x = layers.GlobalAveragePooling2D()(x)  # Global average pooling layer
x = tf.keras.layers.Dropout(0.2)(x)  # Dropout layer with dropout rate of 0.2
outputs = tf.keras.layers.Dense(num_classes)(x)  # Dense layer for classification
model = tf.keras.Model(inputs, outputs)  # Create the final model

# Display the model summary
print(model.summary())

# Compile the model with specified optimizer, loss, and metrics
optimizer = tf.keras.optimizers.Adam(learning_rate=initial_LearningRate)
model.compile(
    optimizer=optimizer,
    loss=[
        tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Sparse categorical cross-entropy loss
    ],
    metrics=[
        tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')  # Sparse categorical accuracy metric
    ]
)

# Save the model after each epoch if it performs better
save_model_after_each_epoch = tf.keras.callbacks.ModelCheckpoint(
    (str(Save_model_at) + "/" + str(Model_name) + "_Split_Part_" + str(selected_split_part) + "_from_" + str(num_split_parts) + '.h5'),
    verbose=1,
    save_best_only=True  # Save only the best model
)

# Define the learning rate reduction callback based on validation loss
lr_reduction = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=Patience,  # Number of epochs with no improvement after which learning rate will be reduced
    verbose=1,
    min_delta=min_delta  # Minimum change in the monitored quantity to qualify as an improvement
)

# Compute class weights based on the training dataset distribution
class_counts = np.bincount(train_generator.classes)  # Count occurrences of each class in the training set
total_samples = sum(class_counts)  # Calculate the total number of samples
class_weights = [total_samples / (len(classes) * count) for count in class_counts]  # Compute class weights
class_weights_dict = {class_label: weight for class_label, weight in enumerate(class_weights)}  # Create a dictionary of class weights

# Check if data balancing is required and print class counts and weights if 'yes'
if str(Give_data_balance).lower() == "yes":
    print(class_counts)
    print(class_weights)

# Train the model using the configured generators and settings
steps_per_epoch = train_generator.samples // train_generator.batch_size  # Calculate steps per epoch
validation_steps = validation_generator.samples // validation_generator.batch_size  # Calculate validation steps

# Define TensorBoard
class CustomTensorBoardCallback_Accuracy(Callback):
    def __init__(self, metrics):
        super(CustomTensorBoardCallback_Accuracy, self).__init__()
        # Initializing the callback with a list of metrics to monitor
        self.metrics = metrics
        # Creating empty lists to store metric values, epochs, and learning rates
        self.metric_values = {metric: [] for metric in metrics}
        self.epochs = []
        self.learning_rates = []

    def on_train_begin(self, logs=None):
        pass  # No action needed when the training starts

    def on_epoch_begin(self, epoch, logs=None):
        # Adding learning rates at the beginning of each epoch
        self.learning_rates.append(self.model.optimizer.lr.numpy())

    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
        self.epochs.append(epoch + 1)  # Incrementing the epoch number by 1
        for metric in self.metrics:
            # Collecting metric values at the end of each epoch
            self.metric_values[metric].append(logs.get(metric))

        # Plotting metrics and learning rates after each epoch
        fig, ax1 = plt.subplots(figsize=(10, 6))  # Creating a plot figure
        lines = []
        colors = ['blue', 'orange', 'green', 'red']  # Colors for different metrics

        # Plotting each metric curve
        for i, metric in enumerate(self.metrics):
            line, = ax1.plot(self.epochs, self.metric_values[metric], '-', color=colors[i], label=metric)
            lines.append(line)

        # Adding a line for the learning rate
        lr_line, = ax1.plot(self.epochs, self.learning_rates, '--', label='Learning Rate')

        # Setting up plot labels, title, ticks, and legend
        ax1.set_ylabel('Value')
        ax1.set_title('Epoch')
        ax1.set_xticks(np.arange(min(self.epochs), max(self.epochs) + 1, 1))
        ax1.set_ylim(0, 1.0)
        ax1.set_yticks(np.arange(0, 1.1, 0.05))
        ax1.legend(lines + [lr_line], self.metrics + ['Learning Rate'])
        ax1.grid(True)
        ax1.tick_params(axis='x', top=True, bottom=False, labeltop=True, labelbottom=False)

        # Adding a secondary y-axis for better readability
        ax2 = ax1.twinx()
        ax2.set_ylim(0, 1.0)
        ax2.set_yticks(np.arange(0, 1.1, 0.05))
        ax2.yaxis.tick_right()
        ax2.yaxis.set_label_position("right")
        ax2.set_ylabel('Value')

        # Adding text labels for curves with an offset
        offset = 0.02
        for i in range(len(self.epochs)):
            x = self.epochs[i]
            y_lr = self.learning_rates[i]
            y_val_loss = self.metric_values['val_loss'][i]
            y_val_acc = self.metric_values['val_accuracy'][i]
            lr_label = f'{y_lr:.1e}'
            lr_label = (lr_label.replace("e-0", "e-").replace("0e", "e"))[2:]
            val_loss_label = str(int(y_val_loss * 100))
            val_acc_label = str(int(y_val_acc * 100))
            ax1.text(x, y_lr - (offset * 4), lr_label, ha='center', va='bottom', color=lr_line.get_color())
            ax1.text(x, y_val_loss - (offset * 3), val_loss_label, ha='center', va='bottom', color=colors[3])
            ax1.text(x, y_val_acc + offset, val_acc_label, ha='center', va='bottom', color=colors[0])
        plt.show()

# List of metrics to monitor during training
metrics = ['val_accuracy', 'accuracy', 'loss', 'val_loss']

# Creating an instance of the CustomTensorBoardCallback_Accuracy class
custom_tensorboard_callback_accuracy = CustomTensorBoardCallback_Accuracy(metrics)

# Train the model with or without data balancing
if str(Give_data_balance).lower() == "yes":
    # Train the model with data balancing and specified callbacks
    model.fit(
        train_generator,
        epochs=Epochs_number,
        steps_per_epoch=steps_per_epoch,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        class_weight=class_weights_dict,  # Apply class weights during training
        callbacks=[save_model_after_each_epoch, lr_reduction, custom_tensorboard_callback_accuracy]
    )
else:
    # Train the model without data balancing and specified callbacks
    model.fit(
        train_generator,
        epochs=Epochs_number,
        steps_per_epoch=steps_per_epoch,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        callbacks=[save_model_after_each_epoch, lr_reduction, custom_tensorboard_callback_accuracy]
    )

# Load the test dataset from the directory using the specified configurations
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    str(test_directory) + "/",
    labels=None,
    shuffle=False,
    image_size=image_size,
    batch_size=batch_size
)

# Make predictions on the test dataset using the trained model
rounded_predictions2 = []
for image in test_dataset:
    predictions = model.predict(image)  # Get predictions for each image in the test dataset
    rounded_predictions = np.argmax(predictions, axis=1)  # Get the class index with the highest probability
    rounded_predictions2.append(rounded_predictions)  # Append the predictions to a list
rounded_predictions2 = np.concatenate(rounded_predictions2)  # Concatenate all predictions into a single array

# Map predicted class indices to their corresponding class names
class_nameslist = []
for i in rounded_predictions2:
    class_nameslist.append(classes[i])  # Map the predicted indices to their respective class names

# Get file paths and true class labels for the test dataset
XX = test_dataset.file_paths
file_pathss = pd.DataFrame(XX, columns=['Path'])  # Create a DataFrame for file paths
unique_word_list = []
for path in XX:
    path = path.replace("/", "\\")
    unique_word = path.split("\\")[-2]  # Extract the true class label from the file path
    unique_word_list.append(unique_word)  # Append true class labels to a list
True_class_lsit = pd.DataFrame(unique_word_list, columns=['True Class'])  # Create a DataFrame for true class labels

# Create DataFrames for predicted and true class labels along with file paths
class_nameslist = pd.DataFrame(class_nameslist, columns=['Predicted Class'])
Classification_result = pd.concat([file_pathss, class_nameslist, True_class_lsit], axis=1)

# Identify misclassified samples by comparing predicted and true class labels
misclassified_result = Classification_result[
    Classification_result['Predicted Class'] != Classification_result['True Class']]

# Save misclassified samples to an Excel file
filename = (str(Save_model_at) + "/" + str(Model_name) + "_Split_Part_" + str(selected_split_part) + "_from_" + str(
    num_split_parts) + "_TestData.xlsx")
writer = pd.ExcelWriter(filename, engine='xlsxwriter')
misclassified_result.to_excel(writer, sheet_name='Misclassified_Test_Data', index=False)  # Write misclassified data to Excel
writer.save()  # Save the Excel file

files.download(filename)  # Download the Excel file
print("************************ Test Dataset classification is completed and saved.")  # Print completion message

FileNotFoundError: [Errno 2] No such file or directory: 'Classification/Decompressed_dataset/Elsafty_RBCs_for_Segmentation_and_Detection_Slide_1 (1)'